### Assignment 4 section 4: Create a recipe generator using RAG

> Add blockquote



```
NOTE: This code should be run with a HuggingFace account with a collab CPU runtime and the option `High Ram`clicked.

```

For this assignment, you'll be creating a RAG-based recipe genertion system using Alibaba's `KingNish/Qwen2.5-0.5b-Test-ft` LLM, a small but high-accuracy LLM that can be run on a CPU. To implement the RAG component, we'll be using the `LlamaIndex` library and, as our embedding model, `BAAI/bge-small-en-v1.5`

This code should be run with a HuggingFace account with a collab CPU runtime and the option `High Ram`clicked.

The recipe dataset we'll be using is `m3hrdadfi/recipe_nlg_lite`. The train split of the dataset will be used for the index and a portion of the test dataset will be used to qualitiatively evaluate the RAG system. You'll also be asked to compare these results qualitatively with vanilla `Qwen2.5-0.5b-Test-ft`, i.e., prompt the `Qwen` base LLM for recipes and compare these to the RAG system to determine if there are any benefits to using RAG for this task.

Your task is to

1. complete the code in each of the cells below by inserting your own code wherevver you see  `### WRITE YOUR CODE HERE ###` Also, add a comment wherever you see `### WRITE YOUR COMMENT HERE ###`  You'll likely want to consult the LlamaIndex documenetation to complete much of this code. The LlamaIndex docs on RAG can be found [here](https://docs.llamaindex.ai/en/stable/understanding/rag/)

2. run all of cells after completing the code and answer the questions at the end of the notebook.



In [ ]:
# uncomment and run in your environment / on Colab, if you haven't installed these packages yet
!pip install llama-index-embeddings-huggingface
!pip install llama-index-llms-huggingface
!pip install sentence-transformers
!pip install datasets
!pip install llama-index
!pip install "transformers[torch]" "huggingface_hub[inference]"
from IPython.display import clear_output
clear_output()

In [ ]:
# import packages
from datasets import load_dataset
import os
import pandas as pd
from llama_index.core import VectorStoreIndex, Settings, Document
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.llms.huggingface import HuggingFaceLLM
from transformers import AutoTokenizer
import torch

In [ ]:
# load dataset from HF
dataset = load_dataset("m3hrdadfi/recipe_nlg_lite")
# convert train split to pandas dataframe
dataset_df = pd.DataFrame(dataset["train"])

In [ ]:
# Let's take a look at the data
dataset_df.head()

In [ ]:
# We'll crate a VectorStorageIndex with the texts from the train dataset. These will be formatted as
#"Name of recipe \n\n ingredients \n\n steps"

texts = [
    f"{row['name']} \n\n {row['ingredients']} \n\n {row['steps']}" for _, row in dataset_df.iterrows()
]
texts[:2]

In [ ]:
#We then load the texts into LlamaIndex's Document object. Later we'll load these into a vector database
documents = [Document(text=t) for t in texts]
documents[0]

In [ ]:
# Create a utility function to format prompts. This will only need to be edited if you use something other than Qwen. If you
# decide to use something other than Qwen, check the HuggingFace model card for that model to determine prompt formatting
def completion_to_prompt(completion):
    return f"{completion}"

In [ ]:
!pip install accelerate # Using `bitsandbytes` 8-bit quantization requires Accelerate
!pip install -U bitsandbytes
clear_output()

In [ ]:
# Save the setting reused by our RAG system across queries and specify the embedding model--
# the model we'll be using is `BAAI/bge-small-en-v1.5` but you're welcome to use another
Settings.embed_model = HuggingFaceEmbedding(
      ### WRITE YOUR CODE HERE ###
)

# pass the LLM to our the settings object
# see these docs for more details: https://docs.llamaindex.ai/en/stable/understanding/using_llms/using_llms/
# and https://docs.llamaindex.ai/en/stable/module_guides/models/llms/usage_custom/
Settings.llm = HuggingFaceLLM(
     ### WRITE YOUR CODE HERE ###
    model_name= ,
    ### WRITE YOUR CODE HERE ###
    tokenizer_name=  ,
    context_window=1024,
    max_new_tokens=128,
    generate_kwargs={"temperature": 0.7, "do_sample": True},
    completion_to_prompt=completion_to_prompt,
    device_map="auto",
    # Explicitly set load_in_8bit=False or remove it -- up to you
    model_kwargs={"torch_dtype": torch.float16, "trust_remote_code": True}, # "load_in_8bit": True
)
print("Set the LLM as KingNish/Qwen2.5-0.5b-Test-ft...")

# Now we create a vector store which converts the documents to Node objects as per
# (https://docs.llamaindex.ai/en/stable/module_guides/indexing/vector_store_index/)
print("Creating index...")
index = VectorStoreIndex.from_documents(
   ### WRITE YOUR CODE HERE ###
                              # note: this may take a while on a CPU so go do something else in the meanwhile
)
print("Done.")

In [ ]:
# https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/
# we define the query engine: generic interface that allows to ask questions over data
query_engine = index.as_query_engine(
    ### WRITE YOUR COMMENT HERE: What does compact do?###
    response_mode="compact",
    ### WRITE YOUR COMMENT HERE: What does similarity_top_k specify?###
    similarity_top_k=3,
    verbose=True,
)
# https://docs.llamaindex.ai/en/stable/module_guides/querying/response_synthesizers/
response = query_engine.query("How do I make creme brulee?")
print(response)

for i, n in enumerate(response.source_nodes):
    print(f"----- Node {i} -----")
    print(n.node.get_content())
    print("score")
    print(n.score)

In [ ]:
# testing loop
rag_responses = []
vanilla_responses = []
retrieved_node_texts = []
retrieved_node_scores = []

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = AutoModelForCausalLM.from_pretrained(
    "KingNish/Qwen2.5-0.5b-Test-ft",
    #device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("KingNish/Qwen2.5-0.5b-Test-ft")

# retrieve 20 random dish names from test dataset to test the system on
test_df = pd.DataFrame(dataset["test"]).sample(20)
test_queries = [f'How do I make creme brulee?']
    #f'How do I make {r["name"]}?' for
    #_, r in test_df.iterrows()
#]
#print(test_queries[:5])

for query in test_queries:# you may want to just run a few of these via [:5]:
    ### WRITE YOUR CODE HERE ###
    # run the query against the RAG system
    response_rag =
    rag_responses.append(str(response_rag))

    # get the texts of the nodes that were retrieved for this query as a list
    retrieved_node_texts.append(
        [### WRITE YOUR CODE HERE ###]
    )

    # get the scores of the texts of the retrieved nodes as a list
    retrieved_node_scores.append(
        [### WRITE YOUR CODE HERE ###]
    )
    ### YOUR CODE HERE ###
    # implement the "vanilla" recipe generator, which simply prompts the base LLM (e.g., Qwen) rather than using Qwen+RAG
    input_text = completion_to_prompt(query)
    input_ids = tokenizer.encode(###WRITE YOUR CODE HERE###).to(device)
    output = model.generate(
          ### WRITE YOUR CODE HERE ###
      )
    prediction = tokenizer.decode(output[0], skip_special_tokens=True)
    vanilla_responses.append(prediction)

In [ ]:
retrieved_node_scores
test_queries#[:5]

In order to answer the questions below, we'll need to examine the output from the RAG and vanilla systems

In [ ]:
print("RAG system results with similarity scores ....")

for i, (texts, scores) in enumerate(zip(retrieved_node_texts, retrieved_node_scores)):
    print("---" * 100)
    print(f"----- Query {i} -----" )
    print(test_queries[i])
    for j, (text, score) in enumerate(zip(texts, scores)):
      print(f"----- Node {j} -----")
      print(text)
      print("score")
      print(score)

In [ ]:
print("RAG and vanilla system responses to each query...")
for i, (rag_response, vanilla_response) in enumerate(zip(rag_responses, vanilla_responses)):
  print("---" * 100)
  print(f"----- Query {i} -----" )
  print(test_queries[i])
  print("----- RAG Response -----")
  print(rag_response)
  print("----- Vanilla Response -----")
  print(vanilla_response.replace(test_queries[i], "").strip())

### Questions

For questions 1 and 2, first qualitatively compare the output of the vanilla and RAG-based systems.


1. Do you observe differences between the quality of the RAG and vanilla responses? If yes, what are these?


2.  Inspect the retrieved recipes and their scores. Do they make sense for the queries? Do the scores match your intuition about their relevance for the query?


3. Was there any benefit to using RAG for this use-case or could we have just used the vanilla system? Are there other use cases where we'd need to use RAG rather than relying on the base weights of an LLM?


4.  What does the embedding model do? What is the measure used to score the relevance of retrieved documents?


